<a href="https://colab.research.google.com/github/NafisaFarah/awesome-remote-sensing-change-detection/blob/master/Baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Useful resources**

- https://github.com/CMU-CREATE-Lab/deep-smoke-machine

**Install packages**

In [ ]:
!git clone --recursive https://github.com/CMU-CREATE-Lab/deep-smoke-machine.git

fatal: destination path 'deep-smoke-machine' already exists and is not an empty directory.


In [ ]:
%cd deep-smoke-machine/back-end/www/optical_flow/
!git submodule update --init --recursive
!git checkout master

/content/deep-smoke-machine/back-end/www/optical_flow
Already on 'master'
Your branch is up to date with 'origin/master'.


**Import libraries**

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

In [ ]:
%cd /content/deep-smoke-machine/back-end/www
from optical_flow.optical_flow import OpticalFlow

/content/deep-smoke-machine/back-end/www


**Mount drive**

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import and process data

I created a small dataset with this other notebook: https://colab.research.google.com/drive/13vYkrLzpgWmVcYiwkwIrBXLM9PvJOASO#scrollTo=k2kHPgo1_7Se

So far the videos are all 3 seconds long.

In [ ]:
# dataframe
df  = pd.read_pickle('/content/drive/MyDrive/AI against Toxic Clouds Shared/05. Subgroup folders/Model builders 2/Notebooks/Mock up data/mockupMetadata')

**Process video**

We extract 12 frames per second, in RGB format with a resolution of 180x180.

In [ ]:
video_directory = '/content/drive/MyDrive/AI against Toxic Clouds Shared/05. Subgroup folders/Model builders 2/Notebooks/Mock up data/Video/'
rgb_directory = '/content/drive/MyDrive/AI against Toxic Clouds Shared/05. Subgroup folders/Model builders 2/Notebooks/Mock up data/RGB/'

In [ ]:
def processVideo(file_name,
                 dir_in ,
                 out_dir):
    rgb_vid_in_p = dir_in + file_name + '.mp4'
    rgb_4d_out_p = out_dir + file_name + '.npy'
    flow_type = None # will not process optical flow
    op = OpticalFlow(rgb_vid_in_p=rgb_vid_in_p, rgb_4d_out_p=rgb_4d_out_p,
                    clip_flow_bound=20, flow_type=flow_type)
    op.process()

In [ ]:
df.apply(lambda x: processVideo(x.file_name, dir_in=video_directory, out_dir =rgb_directory), axis = 1)

In [ ]:
np.load('/content/drive/MyDrive/AI against Toxic Clouds Shared/05. Subgroup folders/Model builders 2/Notebooks/Mock up data/RGB/0-0-2018-05-11-6304-944-6807-1447-180-180-11704-1526076945-1526077120.npy').shape

(36, 180, 180, 3)

# Train, validation, test, split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
dfTrain, dfTest = train_test_split(df, test_size=0.33, random_state=42)

In [ ]:
dfTrain, dfValidation = train_test_split(dfTrain, test_size=0.33, random_state=42)

In [ ]:
%cd '/content'
dfTrain.reset_index(drop=True).to_json('train.json')
dfTest.reset_index(drop=True).to_json('test.json')
dfValidation.reset_index(drop=True).to_json('validation.json')

/content


In [ ]:
pd.read_json('/content/train.json')

,camera_id,file_name,id,label_state,label_state_admin,start_time,url_part,url_root,view_id
0,0,0-1-2019-04-07-6007-928-6509-1430-180-180-9254...,89515,23,-1,2019-04-07 19:25:00,2019-04-07/0-1/0-1-2019-04-07-6007-928-6509-14...,https://smoke.createlab.org/videos/180/,1
1,0,0-1-2018-06-11-6007-1008-6509-1510-180-180-523...,354,23,-1,2018-06-11 12:42:20,2018-06-11/0-1/0-1-2018-06-11-6007-1008-6509-1...,https://smoke.createlab.org/videos/180/,1
2,0,0-8-2018-11-10-3012-1045-3515-1548-180-180-429...,50110,23,-1,2018-11-10 14:39:15,2018-11-10/0-8/0-8-2018-11-10-3012-1045-3515-1...,https://smoke.createlab.org/videos/180/,8
3,0,0-6-2019-03-14-3981-1004-4484-1507-180-180-913...,55478,23,-1,2019-03-14 19:28:05,2019-03-14/0-6/0-6-2019-03-14-3981-1004-4484-1...,https://smoke.createlab.org/videos/180/,6
4,0,0-4-2019-04-01-4897-954-5400-1457-180-180-1122...,81280,23,-1,2019-04-01 23:28:30,2019-04-01/0-4/0-4-2019-04-01-4897-954-5400-14...,https://smoke.createlab.org/videos/180/,4
...,...,...,...,...,...,...,...,...,...
84,0,0-6-2019-02-02-3981-1004-4484-1507-180-180-822...,19225,23,-1,2019-02-02 19:00:30,2019-02-02/0-6/0-6-2019-02-02-3981-1004-4484-1...,https://smoke.createlab.org/videos/180/,6
85,0,0-0-2018-06-11-6304-964-6807-1467-180-180-1394...,297,23,-1,2018-06-12 00:50:20,2018-06-11/0-0/0-0-2018-06-11-6304-964-6807-14...,https://smoke.createlab.org/videos/180/,0
86,0,0-2-2018-07-07-5648-1004-6150-1506-180-180-598...,35476,23,-1,2018-07-07 15:20:35,2018-07-07/0-2/0-2-2018-07-07-5648-1004-6150-1...,https://smoke.createlab.org/videos/180/,2
87,0,0-10-2019-07-26-2614-1123-3116-1625-180-180-99...,99834,19,-1,2019-07-26 20:49:35,2019-07-26/0-10/0-10-2019-07-26-2614-1123-3116...,https://smoke.createlab.org/videos/180/,10


# Fine-tune pre-trained model with W&B monitoring

In [ ]:
!pip install pillow==6.0.0
!pip install torchviz

In [ ]:
%cd /content/deep-smoke-machine/back-end/www
from i3d_learner import I3dLearner

/content/deep-smoke-machine/back-end/www


In [ ]:
!pip install wandb

In [ ]:
!wandb login

wandb: Currently logged in as: michelesergiopozzi (toxic-clouds-model-builders-2). Use `wandb login --relogin` to force relogin


In [ ]:
import wandb
import random

In [ ]:
# Import I3D
from i3d_learner import I3dLearner

In [ ]:
model_params = {'use_cuda':None, # use cuda or not
            'use_tsm':False, # use the Temporal Shift module or not
            'use_nl':False, # use the Non-local module or not
            'use_tc':True, # use the Timeception module or not
            'use_lstm':False, # use LSTM module or not
            'freeze_i3d':True, # freeze i3d layers when training Timeception
            'batch_size_train':8, # size for each batch for training
            'batch_size_test':50, # size for each batch for testing
            'batch_size_extract_features':40, # size for each batch for extracting features
            'max_steps':100, # total number of steps for training
            'num_steps_per_update':2, # gradient accumulation (for large batch size that does not fit into memory)
            'init_lr':0.00000000000000000000000000000001, # initial learning rate
            'weight_decay':0.000001, # L2 regularization
            'momentum':0.9, # SGD parameters
            'milestones':[500, 1500], # MultiStepLR parameters
            'gamma':0.1, # MultiStepLR parameters
            'num_of_action_classes':2, # currently we only have two classes (0 and 1, which means no and yes)
            'num_steps_per_check':50, # the number of steps to save a model and log information
            'parallel':True, # use nn.DistributedDataParallel or not
            'augment':True, # use data augmentation or not
            'num_workers':12, # number of workers for the dataloader
            'mode':"rgb", # can be "rgb" or "flow" or "rgbd"
            'p_frame':"/content/drive/MyDrive/AI against Toxic Clouds Shared/05. Subgroup folders/Model builders 2/Notebooks/Mock up data/RGB/", # path to load video frames
            'code_testing':False # a special flag for testing if the code works
                }

fit_params = {'p_model':'/content/deep-smoke-machine/back-end/data/pretrained_models/RGB-I3D-S3.pt', # the path to load the pretrained or previously self-trained model
            'model_id_suffix':"baseline-0", # the suffix appended after the model id
            'p_metadata_train':"/content/train.json", # metadata path (train)
            'p_metadata_validation':"/content/validation.json", # metadata path (validation)
            'p_metadata_test':"/content/test.json", # metadata path (test)
            'save_model_path':"/content/save_model/", # path to save the models ([model_id] will be replaced)
            'save_tensorboard_path':"/content/tensorboard_path/", # path to save data ([model_id] will be replaced)
            'save_log_path':"/content/log/train.log", # path to save log files ([model_id] will be replaced)
            'save_metadata_path':"/content/" # path to save metadata ([model_id] will be replaced)
}

In [ ]:
model = I3dLearner(**model_params)

# Finetune the RGB-TC model from the RGB-I3D model
model.fit(**fit_params)


INFO:/content/log/train.log0:============================================================
INFO:/content/log/train.log0:============================================================
INFO:/content/log/train.log0:Use Two-Stream Inflated 3D ConvNet learner
INFO:/content/log/train.log0:save_model_path: /content/save_model/
INFO:/content/log/train.log0:save_tensorboard_path: /content/tensorboard_path/
INFO:/content/log/train.log0:save_log_path: /content/log/train.log0
INFO:/content/log/train.log0:p_metadata_train: /content/train.json
INFO:/content/log/train.log0:p_metadata_validation: /content/validation.json
INFO:/content/log/train.log0:p_metadata_test: /content/test.json
INFO:/content/log/train.log0:
Parameters:
  use_tsm: False
  use_nl: False
  use_tc: True
  use_lstm: False
  freeze_i3d: True
  batch_size_train: 8
  batch_size_test: 50
  batch_size_extract_features: 40
  max_steps: 100
  num_steps_per_update: 2
  init_lr: 1e-32
  weight_decay: 1e-06
  momentum: 0.9
  milestones: [500, 15

Use Two-Stream Inflated 3D ConvNet learner
save_model_path: /content/save_model/
save_tensorboard_path: /content/tensorboard_path/
save_log_path: /content/log/train.log0
p_metadata_train: /content/train.json
p_metadata_validation: /content/validation.json
p_metadata_test: /content/test.json

Parameters:
  use_tsm: False
  use_nl: False
  use_tc: True
  use_lstm: False
  freeze_i3d: True
  batch_size_train: 8
  batch_size_test: 50
  batch_size_extract_features: 40
  max_steps: 100
  num_steps_per_update: 2
  init_lr: 1e-32
  weight_decay: 1e-06
  momentum: 0.9
  milestones: [500, 1500]
  gamma: 0.1
  num_of_action_classes: 2
  num_steps_per_check: 50
  parallel: True
  augment: True
  num_workers: 12
  mode: rgb
  p_frame: /content/drive/MyDrive/AI against Toxic Clouds Shared/05. Subgroup folders/Model builders 2/Notebooks/Mock up data/RGB/

Initialize the I3D+Timeception model...
num_tc_layers: 1
freeze_i3d: True
Input size:
	 torch.Size([1, 3, 36, 224, 224])
Freeze I3D model


INFO:/content/log/train.log0:Load model weights from /content/deep-smoke-machine/back-end/data/pretrained_models/RGB-I3D-S3.pt


I3D model output size:
	 torch.Size([1, 1024, 5, 7, 7])
Timeception model output size:
	 torch.Size([1, 1280, 2, 7, 7])
Final layer output size:
	 torch.Size([1, 400, 1])
Load model weights from /content/deep-smoke-machine/back-end/data/pretrained_models/RGB-I3D-S3.pt


INFO:/content/log/train.log0:Weights were from nn.DataParallel or DistributedDataParallel...
INFO:/content/log/train.log0:Remove 'module.' prefix from state_dict keys...
INFO:/content/log/train.log0:Load model weights from /content/deep-smoke-machine/back-end/data/pretrained_models/RGB-I3D-S3.pt
  warnings.warn('Saturation jitter enabled. Will slow down loading immensely.')

  warnings.warn('Hue jitter enabled. Will slow down loading immensely.')

INFO:/content/log/train.log0:Create dataloader for train
  warnings.warn(_create_warning_msg(

INFO:/content/log/train.log0:Create dataloader for validation


Weights were from nn.DataParallel or DistributedDataParallel...
Remove 'module.' prefix from state_dict keys...
Load model weights from /content/deep-smoke-machine/back-end/data/pretrained_models/RGB-I3D-S3.pt
Delete logits in the I3D model...
Create dataloader for train
Create dataloader for validation


INFO:/content/log/train.log0:----------------------------------------
INFO:/content/log/train.log0:phase train
INFO:/content/log/train.log0:epochs: 1 steps: 0/100


----------------------------------------
phase train
epochs: 1 steps: 0/100


  0%|          | 0/2 [00:00<?, ?it/s]


KeyError: ignored

In [ ]:
model = I3dLearner(
    mode="rgb",
    augment=True,
    use_tc=True,
    freeze_i3d=True,
    batch_size_train=8,
    milestones=[1000, 2000],
    num_steps_per_update=1,
    p_frame='/content/drive/MyDrive/AI against Toxic Clouds Shared/05. Subgroup folders/Model builders 2/Notebooks/Mock up data/RGB/')

# Finetune the RGB-TC model from the RGB-I3D model
model.fit(
    p_model='/content/deep-smoke-machine/back-end/data/pretrained_models/RGB-I3D-S3.pt',
    model_id_suffix="-s3",
    p_metadata_train="/content/train.json",
    p_metadata_validation="/content/validation.json",
    p_metadata_test="/content/test.json")

INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:============================================================
INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:============================================================
INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:Use Two-Stream Inflated 3D ConvNet learner
INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:save_model_path: ../data/saved_i3d/6b584aa-i3d-rgb-s3/model/
INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:save_tensorboard_path: ../data/saved_i3d/6b584aa-i3d-rgb-s3/run/
INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:save_log_path: ../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0
INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:p_metadata_train: /content/train.json
INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:p_metadata_validation: /content/validation.json
INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:p_metadata_test: /content/test.json
INFO:..

Use Two-Stream Inflated 3D ConvNet learner
save_model_path: ../data/saved_i3d/6b584aa-i3d-rgb-s3/model/
save_tensorboard_path: ../data/saved_i3d/6b584aa-i3d-rgb-s3/run/
save_log_path: ../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0
p_metadata_train: /content/train.json
p_metadata_validation: /content/validation.json
p_metadata_test: /content/test.json

Parameters:
  use_tsm: False
  use_nl: False
  use_tc: True
  use_lstm: False
  freeze_i3d: True
  batch_size_train: 8
  batch_size_test: 50
  batch_size_extract_features: 40
  max_steps: 2000
  num_steps_per_update: 1
  init_lr: 0.1
  weight_decay: 1e-06
  momentum: 0.9
  milestones: [1000, 2000]
  gamma: 0.1
  num_of_action_classes: 2
  num_steps_per_check: 50
  parallel: True
  augment: True
  num_workers: 12
  mode: rgb
  p_frame: /content/drive/MyDrive/AI against Toxic Clouds Shared/05. Subgroup folders/Model builders 2/Notebooks/Mock up data/RGB/

Initialize the I3D+Timeception model...
num_tc_layers: 1
freeze_i3d: True
Input s

INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:Load model weights from /content/deep-smoke-machine/back-end/data/pretrained_models/RGB-I3D-S3.pt
INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:Weights were from nn.DataParallel or DistributedDataParallel...
INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:Remove 'module.' prefix from state_dict keys...
INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:Load model weights from /content/deep-smoke-machine/back-end/data/pretrained_models/RGB-I3D-S3.pt


I3D model output size:
	 torch.Size([1, 1024, 5, 7, 7])
Timeception model output size:
	 torch.Size([1, 1280, 2, 7, 7])
Final layer output size:
	 torch.Size([1, 400, 1])
Load model weights from /content/deep-smoke-machine/back-end/data/pretrained_models/RGB-I3D-S3.pt
Weights were from nn.DataParallel or DistributedDataParallel...
Remove 'module.' prefix from state_dict keys...
Load model weights from /content/deep-smoke-machine/back-end/data/pretrained_models/RGB-I3D-S3.pt


  warnings.warn('Saturation jitter enabled. Will slow down loading immensely.')

  warnings.warn('Hue jitter enabled. Will slow down loading immensely.')

INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:Create dataloader for train
  warnings.warn(_create_warning_msg(

INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:Create dataloader for validation
INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:----------------------------------------
INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:phase train
INFO:../data/saved_i3d/6b584aa-i3d-rgb-s3/log/train.log0:epochs: 1 steps: 0/2000


Delete logits in the I3D model...
Create dataloader for train
Create dataloader for validation
----------------------------------------
phase train
epochs: 1 steps: 0/2000


  0%|          | 0/2 [00:00<?, ?it/s]


KeyError: ignored

In [ ]:




# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="baseline-model-I3D",

    # track hyperparameters and run metadata
    config={
    "fit_params": fit_params,
    "model_params": model_params,
    "data": "dummy-dataset"
    }
)

model = I3dLearner(**model_params)

# Finetune the RGB-TC model from the RGB-I3D model
model.fit(**fit_params)
    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})


    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

SyntaxError: ignored

## Fine-tuning, monitoring with weight and bias